## Chapter 09 / Problem 11 - Implement and inorder traversal with O(1) space

### Problem Statement

The direct implementation of an inorder traversal using recursion has $ O(h) $ space complexity, where **h** is the height of the tree.  Recursion can be removed with an explicit stack, but the space complexity remains $ O(h) $.

Write a nonrecursive program for computing the inorder traversal sequence for a binary tree.  Assume nodes have parent fields.

***Hint:*** *How can you tell whether a node is a left child or right child of its parent?*

###  Subclassing binarytree.Node

I like being able to show a diagram of my tree, and [binarytree](https://anaconda.org/conda-forge/binarytree) provides that for us.  

However, we need to subclass Node, so that there is also a **parent** pointer for us to work with.

In [1]:
from binarytree import Node


class PNodeTypeError(Exception):
    """Wrong type error"""


class PNode(Node):
    """Subclass of binaryTree.Node"""
    def __init__(self, value, left=None, right=None, parent=None):
        Node.__init__(self, value, left=left, right=right)
        self.parent = parent

    def __setattr__(self, attr, obj):
        Node.__setattr__(self, attr, obj)
        if attr in ['left', 'right'] and obj is not None:
            obj.parent = self
        elif attr == 'parent':
            if obj is not None and not isinstance(obj, PNode):
                raise PNodeTypeError('Must set parent to a PNode or None')

### Now let's build our tree

In [2]:
tree = PNode(5)
tree.left = PNode(2)
tree.right = PNode(7)
tree.right.right = PNode(8)
tree.right.left = PNode(6)
tree.left.right = PNode(3)
tree.left.right.right = PNode(4)
tree.left.left = PNode(1)

### Let's visualize the tree

In [3]:
print(tree)


    ____5__
   /       \
  2         7
 / \       / \
1   3     6   8
     \
      4



### So an inorder traversal goes like this

1.  Traverse the left-subtree (recursive)
2.  Traverse the root
3.  Traverse the right-subtree (recursive)

So in our example, we want the following traversal

```1 - 2 - 3 - 4 - 5 - 6 - 7 - 8```

But we **cannot use recursion**, nor can we use an explicit stack.

So here's the approach, using a **trailing object reference**
### Until you have completely traversed the tree
#### if your previous node was a PARENT
1.  if you can go to the left
    * go to the left
2.  if you cannot go to the left
    * write out the value
    * go to the first available of right or parent


####  else if your previous node was a LEFT
1.  write out the value
2.  go to the first available of right or parent


####  for anything else (i.e. previous node was RIGHT)
1.  go the parent



In [4]:
def solve(root):
    """This makes use of the special *or* and *is* functions
        *or* is used to get the first non-None value
        *is* is used for object memory address equality
    """
    prev, result = None, []
    while root:
        if prev is root.parent:             # last node was the parent
            if root.left:                       
                next = root.left                    
            else:                               
                result.append(root.value)           
                next = root.right or root.parent    
        elif prev is root.left:             # last node was left
            result.append(root.value)           
            next = root.right or root.parent        
        elif prev is root.right:            # last node was right
            next = root.parent                  

        # keep track of previously visited node
        prev, root = root, next
        
    return result

def report(label, tree):
    print(f"{label}:\n{tree}\n traversal: {solve(tree)}")

In [5]:
report("example tree", tree)

example tree:

    ____5__
   /       \
  2         7
 / \       / \
1   3     6   8
     \
      4

 traversal: [1, 2, 3, 4, 5, 6, 7, 8]


### Let's try a narrow tree that slants to the left

In [6]:
tree = PNode(1)
tree.left = PNode(2)
tree.left.left = PNode(3)
tree.left.left.left = PNode(4)
tree.left.left.left.left = PNode(5)

report("left-leaning tree", tree)

left-leaning tree:

        1
       /
      2
     /
    3
   /
  4
 /
5

 traversal: [5, 4, 3, 2, 1]


### Left-leaning results

You can see that the left-most nodes come first in our inorder traversal.

### Let's try a narrow tree that slants to the right

In [7]:
tree = PNode(1)
tree.right = PNode(2)
tree.right.right = PNode(3)
tree.right.right.right = PNode(4)
tree.right.right.right.right = PNode(5)

report("right-leaning tree", tree)

right-leaning tree:

1
 \
  2
   \
    3
     \
      4
       \
        5

 traversal: [1, 2, 3, 4, 5]


### Right-leaning results

You can see that the left-most nodes **still** come first in our inorder traversal.

### Let's try a jagged tree

In [8]:
tree = PNode(1)
tree.left = PNode(2)
tree.left.right = PNode(3)
tree.left.right.left = PNode(4)
tree.left.right.left.right = PNode(5)
tree.right = PNode(6)

report("jagged tree", tree)

jagged tree:

  ______1
 /       \
2____     6
     \
    __3
   /
  4
   \
    5

 traversal: [2, 4, 5, 3, 1, 6]


### Jagged results

**Yet again** the left-most nodes **still** come first in our inorder traversal.

We can visually see the pattern that will help us remember inorder traversals.

### Complexity Analysis

| Feature           | Complexity      |
|  :-----------:    | :------------:  |
|  Time             | $ O(n) $        |
|  Additional Space | $ O(1) $        |